This script attempts to predict a Bob Graham round finishing time based on the race results of other finishers. The membership list of the BGR is imported into the DUV ultramarathon databased to obtain runner ID's. The each race result is then scraped from the DUV site and the results of any of the Bob Graham finishers are then stored. 

The idea is then to input a specific race result that you've completed or specific distance stats to see where you compare to other finishers.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL pointing to the CSV data
CSV_URL = "http://bobgrahamclub.org.uk/api/data_bgr_listing.csv"

bg_results = pd.read_csv(CSV_URL)
print(bg_results.head()) 


        gn        fn gender  age   time        date  weekNum dir  mem  orig  \
0     Alan    Heaton      M   32  22:18  1960-06-26       25   A    1     0   
1  Stanley  Bradshaw      M   48  23:25  1960-07-09       27   C    2     0   
2  Kenneth    Heaton      M   35  22:13  1961-06-24       25   C    3     0   
3     Eric     Beard      M   31  23:35  1963-07-06       27   C    4     0   
4     Joss    Naylor      M   34  23:37  1971-06-26       25   C    5     0   

   ...  opt  record  lr  died  nationality                       club prev  \
0  ...    0       1   0  2019      British  Clayton-le-Moors Harriers    0   
1  ...    0       0   0  2010      British  Clayton-le-Moors Harriers    1   
2  ...    0       1   0  2014      British  Clayton-le-Moors Harriers    0   
3  ...    0       0   0  1969      British        Leeds Athletic Club    0   
4  ...    1       0   0     0      British       Kendal Athletic Club    0   

   postcode                                             

In [2]:
# Load DUV bulk search
DUV_PATH = "./data/DUV_Bulkexport_20240416.xls"

def load_html_to_dataframe(file_path):
    try:
        dfs = pd.read_html(file_path, encoding='utf-8')
        return dfs
    except Exception as e:
        print(f"Failed to load the HTML file: {e}")
        return None

data_frames = load_html_to_dataframe(DUV_PATH)
if data_frames:
    for i, df in enumerate(data_frames):
        print(f"Table {i+1}:")
        print(df.head())  # Display the first few rows of each DataFrame

Table 1:
  Unnamed: 0_level_0 DUV Ultra Marathon Statistics                       \
             Counter                       Surname First name Runner ID   
0                  1                        Heaton       Alan   1462093   
1                  2                      Bradshaw    Stanley    Search   
2                  3                        Heaton    Kenneth    Search   
3                  4                         Beard       Eric   1048701   
4                  5                        Naylor       Joss    Search   

                                                                
  Original name Nat.  M/F     YOB Date of birth Cat. internat.  
0           NaN  GBR    M     NaN    00.00.0000            NaN  
1           NaN  NaN  NaN     NaN           NaN            NaN  
2           NaN  NaN  NaN     NaN           NaN            NaN  
3           NaN  GBR    M  1931.0    00.00.0000            NaN  
4           NaN  NaN  NaN     NaN           NaN            NaN  


In [3]:
valid_runners = df[df[('DUV Ultra Marathon Statistics', 'Runner ID')] != 'Search']
valid_runners.columns = valid_runners.columns.droplevel(0)
valid_runners.head()

,Counter,Surname,First name,Runner ID,Original name,Nat.,M/F,YOB,Date of birth,Cat. internat.
0,1,Heaton,Alan,1462093,NaN,GBR,M,NaN,00.00.0000,NaN
3,4,Beard,Eric,1048701,NaN,GBR,M,1931.0,00.00.0000,NaN
6,7,Talbot,Donald,482320,NaN,GBR,M,1932.0,00.00.0000,NaN
12,13,Weir,Dennis,113489,NaN,GBR,M,1938.0,25.03.1938,M85
13,14,Millen,Boyd,1051631,NaN,GBR,M,1936.0,25.01.1936,M85


In [28]:
""" def fetch_event_data(url):
    """ Fetch and parse individual event data from a table with a specific ID. """
    try:
        response = requests.get(url)
        if response.status_code == 200:
            # Use pandas to read the specific table by its ID
            df_list = pd.read_html(response.content, attrs={'id': 'Resultlist'})
            return df_list[0] if df_list else None
        else:
            print(f"Failed to fetch {url}: HTTP {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching event data from {url}: {e}")
        return None
    
def fetch_html(url):
    """ Fetch the HTML content of a given URL. """
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    else:
        print(f"Failed to fetch {url}: HTTP {response.status_code}")
        return None

def extract_page_count(soup):
    """ Extract the total number of pages from pagination. """
    pagination = soup.find('div', class_='pagination')
    if pagination:
        last_link = pagination.find_all('a')[-2]  # assuming the second last link is the last page
        last_page_number = int(last_link.get_text())
        return last_page_number
    return 1

def extract_event_links(base_url, soup):
    """ Extract event links from the page. """
    links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if 'getresultevent.php?event=' in href:
            links.append(base_url + href)
    return links

def main():
    base_url = "https://statistik.d-u-v.org/"
    start_year = 2024
    end_year = 2024  # Adjust as needed

    for year in range(start_year, end_year + 1):
        page_url = f"{base_url}geteventlist.php?year={year}&dist=all&country=all&surface=all&sort=1&page=1"
        first_page = fetch_html(page_url)
        if first_page:
            num_pages = extract_page_count(first_page)
            all_data = []

            for page in range(1, num_pages + 1):
                page_url = f"{base_url}geteventlist.php?year={year}&dist=all&country=all&surface=all&sort=1&page={page}"
                page_soup = fetch_html(page_url)
                if page_soup:
                    event_urls = extract_event_links(base_url, page_soup)
                    for url in event_urls:
                        event_data = fetch_event_data(url)
                        if event_data is not None:
                            all_data.append(event_data)

            if all_data:
                final_df = pd.concat(all_data, ignore_index=True)
                final_df.to_csv(f'all_events_data_{year}.csv', index=False)
                print(f"Saved all event data for {year} to 'all_events_data_{year}.csv'.")
            else:
                print(f"No data was extracted for {year}.")

if __name__ == "__main__":
    main() """

In [4]:
def fetch_html(url):
    """Fetch the content of a URL."""
    response = requests.get(url)
    return BeautifulSoup(response.content, 'html.parser') if response.ok else None

def extract_page_count(soup):
    """Extract the number of pages from the pagination element."""
    pagination = soup.find('div', class_='pagination')
    return int(pagination.find_all('a')[-2].text) if pagination else 1

def extract_event_links(base_url, soup):
    """ Extract event links from the page. """
    links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if 'getresultevent.php?event=' in href:
            links.append(base_url + href)
    return links

def extract_event_details(soup):
    """Extract 'Event', 'Date', and 'Distance' from the HTML content."""
    details = {}
    try:
        info_rows = soup.find_all('tr')  # Find all table rows in the page
        for row in info_rows:
            # Look for rows where the first cell contains the labels we're interested in
            header_cell = row.find('td')
            if header_cell and header_cell.find('b'):  # Check for bold tags which might contain labels
                label = header_cell.get_text(strip=True).rstrip(':')
                value_cell = header_cell.find_next_sibling('td')  # Get the next sibling cell for the value
                if label in ['Date', 'Event', 'Distance'] and value_cell:
                    details[label] = value_cell.get_text(strip=True)
    except Exception as e:
        print(f"Error extracting event details: {e}")
    return details


def fetch_event_data(table_soup, event_details):
    """Extract data and runner IDs from the event table, including event details."""
    data = []
    headers = [th.text.strip() for th in table_soup.find_all('th')]
    # Append the event details headers
    headers.extend(['Runner ID', 'Event', 'Date', 'Distance'])

    rows = table_soup.find_all('tr')[1:]  # Skip the header row
    for row in rows:
        cols = row.find_all('td')
        row_data = [col.text.strip() for col in cols]
        # Get runner ID
        link = cols[2].find('a', href=True)  # Assuming the third column has the link
        runner_id = link['href'].split('runner=')[-1] if link else 'No ID'
        # Include event details and runner ID
        row_data.extend([runner_id, event_details['Event'], event_details['Date'], event_details['Distance']])
        data.append(row_data)

    return pd.DataFrame(data, columns=headers)

def fetch_event_data(table_soup, event_details, valid_ids):
    """Extract data and runner IDs from the event table, appending event details, filtered by valid runner IDs."""
    data = []
    headers = [th.text.strip() for th in table_soup.find_all('th')]
    headers.extend(['Runner ID', 'Event', 'Date', 'Distance'])
    rows = table_soup.find_all('tr')[1:]  # Skip header row
    
    for row in rows:
        cols = row.find_all('td')
        if len(cols) > 2:
            link = cols[2].find('a', href=True)
            if link and 'runner=' in link['href']:
                runner_id = link['href'].split('runner=')[-1]
                # Only add data if runner_id is in the list of valid_ids
                if runner_id in valid_ids:
                    row_data = [col.text.strip() for col in cols]
                    row_data.extend([runner_id, event_details.get('Event', 'N/A'), event_details.get('Date', 'N/A'), event_details.get('Distance', 'N/A')])
                    data.append(row_data)
    
    return pd.DataFrame(data, columns=headers)

def scrape_events(base_url, event_url):
    """Scrape events with additional details."""
    soup = fetch_html(base_url + event_url)
    event_details = extract_event_details(soup)
    table_soup = soup.find('table', {'id': 'Resultlist'})
    if table_soup:
        return fetch_event_data(table_soup, event_details)
    return pd.DataFrame()


# URL configuration
base_url = "https://statistik.d-u-v.org/"
event_url = "getresultevent.php?event=102988"

# Perform the scraping
#event_data = scrape_events(base_url, event_url)

# Display or export the data
#ßprint(event_data)


In [7]:
def main():
    base_url = "https://statistik.d-u-v.org/"
    start_year = 2020
    end_year = 2024  # Adjust as needed
    valid_ids = set(valid_runners["Runner ID"])

    for year in range(start_year, end_year + 1):
        page_url = f"{base_url}geteventlist.php?year={year}&dist=all&country=all&surface=all&sort=1&page=1"
        first_page = fetch_html(page_url)
        if first_page:
            num_pages = extract_page_count(first_page)
            all_data = []

            for page in range(1, num_pages + 1):
                page_url = f"{base_url}geteventlist.php?year={year}&dist=all&country=all&surface=all&sort=1&page={page}"
                page_soup = fetch_html(page_url)
                if page_soup:
                    event_links = extract_event_links(base_url, page_soup)
                    for event_link in event_links:
                        event_page = fetch_html(event_link)
                        if event_page:
                            event_details = extract_event_details(event_page)
                            table_soup = event_page.find('table', {'id': 'Resultlist'})
                            if table_soup:
                                event_data = fetch_event_data(table_soup, event_details, valid_ids)
                                if not event_data.empty:
                                    all_data.append(event_data)

            if all_data:
                final_df = pd.concat(all_data, ignore_index=True)
                final_df.to_csv(f'all_events_data_{year}.csv', index=False)
                print(f"Saved all event data for {year} to 'all_events_data_{year}.csv'.")
            else:
                print(f"No data was extracted for {year}.")

if __name__ == "__main__":
    main()


0       1462093
3       1048701
6        482320
12       113489
13      1051631
         ...   
2858    1361288
2860    1168335
2861     653340
2862     861254
2863     995441
Name: Runner ID, Length: 1092, dtype: object